In [1]:
from TPMetrics import TPMetrics
import configparser
import os
import pandas as pd
import numpy as np
import re
from itertools import repeat

In [6]:
tpm = TPMetrics(r'/home/rafael/CNIC/Metabolomics/TurboPutativeCode/testing/std_test_TPMetric')
tpm.getClasses()
tpm.getCorrelations()
#tpm.writeOutfile()